In [1]:
from tqdm import tqdm
import numpy as np
from itertools import combinations

## Part 1

In [2]:
# checks if a position [y x] is within the edges of a n_rows x n_cols array
def is_in_map(current_pos, n_rows, n_cols):
    return (current_pos[0]>=0 and current_pos[0]<n_rows) and (current_pos[1]>=0 and current_pos[1]<n_cols)

In [3]:
# checks if two tuples [np.array([y,x]) , [a, b]] have the same content, so we do not add already visited beams  
def is_same_beam(beam1, beam2):
    return all(beam1[0] == beam2[0]) and beam1[1] == beam2[1]

In [29]:
# given a beam [np.array([y,x]) , [a, b]] it follows it 
# returns the corresponding energized map and the eventual beams generated by splitting
def follow_beam(total_map, new_beam):
    n_rows = len(total_map)
    n_cols = len(total_map[0])
    energized_map = [[0 for xdx in range(n_cols)] for ydx in range(n_rows)]
    current_pos = np.array(new_beam[0])
    current_direction = new_beam[1]
    extra_beams = []
    
    # as long as we are in the map
    while is_in_map(current_pos, n_rows, n_cols):
        # we energize the current spot
        energized_map[current_pos[0]][current_pos[1]] =1
        
        # depending on the type of tile
        match total_map[current_pos[0]][current_pos[1]]:
            case '.':
                pass
            case '-': 
                # if we split the beam, we return the two split beams
                if current_direction[0] != 0:
                    extra_beams.append([current_pos,[0,1] ])
                    extra_beams.append([current_pos,[0,-1] ])
                    return energized_map, extra_beams
            case '|':
                # if we split the beam, we return the two split beams
                if current_direction[1] != 0:
                    extra_beams.append([current_pos,[1,0] ])
                    extra_beams.append([current_pos,[-1,0] ])
                    return energized_map, extra_beams
            case '/':
                # we change direction
                match current_direction:
                    case [1,0]:
                        current_direction = [0,-1]
                    case [-1,0]:
                        current_direction = [0,1]
                    case [0,1]:
                        current_direction = [-1, 0]
                    case [0,-1]:
                        current_direction = [1,0]
                    
            case '\\':
                # we change direction
                match current_direction:
                    case [1,0]:
                        current_direction = [0,1]
                    case [-1,0]:
                        current_direction = [0,-1]
                    case [0,1]:
                        current_direction = [1, 0]
                    case [0,-1]:
                        current_direction = [-1,0] 

        # updated current_pos following the new direction
        current_pos = current_pos + current_direction


    return energized_map, extra_beams

In [40]:
# given a map and a starting beam, computes all the energized path from all the split beams
def compute_energized_map_from_edge(total_map, new_beam):
    n_rows = len(total_map)
    n_cols = len(total_map[0])
    total_energized_map = np.array([[0 for xdx in range(n_cols)] for ydx in range(n_rows)])


    starting_points_directions = [new_beam]
    beams_already_followed = []

    # as long as we have beams to follow
    while starting_points_directions:
        # remove first beam
        new_beam = starting_points_directions.pop()
        # obtain energized path, new beams splitting
        new_beam_map, extra_beams = follow_beam(total_map, new_beam)
        # add new_beam to the beams already explored
        beams_already_followed += [new_beam]
        
        # adds to starting_point_directions only the beams that have not been explored yet
        for beam in extra_beams:
            is_old_beam = False
            for old_beam in beams_already_followed:
                if is_same_beam(beam, old_beam):
                    is_old_beam = True
                    break
            if not is_old_beam:
                starting_points_directions += [beam]

        total_energized_map += new_beam_map
    
        
    return total_energized_map

In [41]:
total_map = []
# generate map 
with open('Day16_input.txt') as f:
    for line in f:
        line = list(line.strip('\n') )
        total_map.append(line)

total_energized_map = compute_energized_map_from_edge(total_map, [np.array([0,0]), [0,1]])

print(sum(sum(total_energized_map >0)))


6361


## Part 2

In [42]:
total_map = []
# generate map 
with open('Day16_input.txt') as f:
    for line in f:
        line = list(line.strip('\n') )
        total_map.append(line)


n_rows = len(total_map)
n_cols = len(total_map[0])

# generates all the possible starting points
edges_with_entrance_dir = []
for ydx in range(n_rows):
    edges_with_entrance_dir.append([np.array([ydx,0]), [0,1]])
    edges_with_entrance_dir.append([np.array([ydx,n_cols -1]), [0, -1]])
    
for xdx in range(n_cols):
    edges_with_entrance_dir.append([np.array([0,xdx]), [1,0]])
    edges_with_entrance_dir.append([np.array([n_rows -1, xdx]), [-1,0]])


max_coverage = 0

# for every starting point, computes the number of energized tiles, and keeps the maximum
for beam in tqdm(edges_with_entrance_dir):
    total_energized_map = compute_energized_map_from_edge(total_map, beam)
    
    coverage = sum(sum(total_energized_map >0))
    if coverage > max_coverage:
        max_coverage = coverage
        
print(max_coverage)

100%|████████████████████████████████████████████████████████████████████████████████| 440/440 [02:12<00:00,  3.32it/s]

6701
